## Limpeza e Engenharia de Features - Clima de Brasília em 2024

### Introdução e Objetivo

A qualidade de qualquer análise de dados ou modelo de machine learning está ligada à qualidade dos dados de entrada. Dados brutos, repletos de inconsistências, valores ausentes ou formatos inadequados, podem levar a conclusões equivocadas e modelos de baixo desempenho.

O objetivo principal deste notebook é, portanto, realizar o pré-processamento completo dos dados brutos do clima de Brasília. Este processo é a fundação de todo o projeto, englobando as etapas de limpeza, tratamento de valores ausentes e, crucialmente, a engenharia de novas features (variáveis) para enriquecer o dataset.

O resultado final será um conjunto de dados limpo, robusto e estruturado, pronto para ser utilizado na fase subsequente de Análise Exploratória de Dados (EDA) e posteriormente, na modelagem.

**Dados:**

O dataset contém informações diárias de qualidade do ar para Brasília, coletadas durante todo o ano de 2024, a partir do Banco de Dados Meteorológicos para Ensino e Pesquisa (BDMEP) do INMET. Os dados incluem variáveis como concentração de poluentes atmosféricos, temperatura, umidade, precipitação e outros parâmetros meteorológicos relevantes para análise ambiental e estudos de poluição do ar.

### Descrição do Processo

O fluxo de trabalho deste notebook está organizado nas seguintes etapas principais:

- Carga e Inspeção Inicial

- Limpeza dos Dados

- Engenharia de Features (Feature Engineering)

### Resultado Esperado

Ao final da execução deste notebook, o DataFrame processado e enriquecido será salvo em um novo arquivo. Este arquivo servirá como a fonte de dados para as análises exploratórias e modelagens subsequentes do projeto, garantindo a reprodutibilidade e a consistência do trabalho.

In [2]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

In [17]:
# Importando os dados e inspeção inicial
df = pd.read_csv("../data/INMET_CO_DF_A001_BRASILIA_01-01-2024_A_31-12-2024.csv", encoding='windows-1252', sep=';')
print(f"Quantidade de linhas: {df.shape[0]}")
print(f"Quantidade de colunas: {df.shape[1]}")

Quantidade de linhas: 8784
Quantidade de colunas: 20


In [18]:
print(f"Resumo Geral:\n")
df.info(memory_usage='deep')

Resumo Geral:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Data                                                   8784 non-null   object 
 1   Hora UTC                                               8784 non-null   object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8760 non-null   object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8760 non-null   object 
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8760 non-null   object 
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8760 non-null   object 
 6   RADIACAO GLOBAL (Kj/m²)                                4704 non-null   object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8758 non-null   object 
 8   TEMPERATURA DO PONTO DE ORVALHO (

In [19]:
df.head(4)

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2024/01/01,0000 UTC,0,"885,7","885,7","885,3",NaN,22,"18,2","22,6","21,8","18,6","18,2",81.0,77.0,79.0,311.0,"2,8","1,5",NaN
1,2024/01/01,0100 UTC,0,"886,7","886,7","885,7",NaN,"21,2","18,6",22,"21,1","18,7","18,1",85.0,79.0,85.0,315.0,"2,9","1,5",NaN
2,2024/01/01,0200 UTC,0,"887,1","887,2","886,7",NaN,"20,9","18,8","21,5","20,9","18,9","18,7",88.0,85.0,88.0,305.0,"3,3","1,1",NaN
3,2024/01/01,0300 UTC,0,"887,1","887,4","887,1",NaN,"20,9","18,3","21,1","20,8","18,9","18,3",88.0,85.0,85.0,304.0,"3,3","1,5",NaN


Nota-se que cada amostra do dia, a coluna `Hora UTC` se move de 100 em 100. Isso demonstra que o dataset possui dados nao especificamente diários, mas sim a cada hora. Outro ponto importante é uma colua completa de valores ausentes e sem nome, que será removida.

In [20]:
# Removendo coluna sem dados
df.drop(columns="Unnamed: 19", inplace=True)

### Transformando colunas de data e hora

Serão criadas novas features de datas para melhorar o modelo a encontrar padrões. Features cíclicas com `sen` e `cos` para o modelo entender que o dia 6 (Domingo) está perto do dia 0 (Segunda), ou que o mês 12 (Dezembro) está perto do mês 1 (Janeiro). A mesma regra será aplicada às horas

In [21]:
# garantir que a data seja o index do dataset
df['data'] = pd.to_datetime(df['Data'])
df.set_index('data', inplace=True)
df.sort_index(inplace=True)

In [22]:
# Criando novas features de data
df["dia_da_semana"] = df.index.dayofweek
df["dia_do_mes"] = df.index.day
df["mes"] = df.index.month
df["semana_do_ano"] = df.index.isocalendar().week

# features cíclicas
df['dia_semana_sin'] = np.sin(2 * np.pi * df['dia_da_semana']/7.0)
df['dia_semana_cos'] = np.cos(2 * np.pi * df['dia_da_semana']/7.0)

df['mes_sin'] = np.sin(2 * np.pi * df['mes']/12.0)
df['mes_cos'] = np.cos(2 * np.pi * df['mes']/12.0)

df.drop(columns=['dia_da_semana', 'mes'], inplace=True)

In [23]:
# Criando features de hora
df['hora_num'] = df['Hora UTC'].str.replace(' UTC', '').astype(int) // 100
df['hora_sin'] = np.sin(2 * np.pi * df['hora_num'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['hora_num'] / 24)

df.drop(columns=['Data', 'Hora UTC'], inplace=True)

In [24]:
# Visualizar o resultado
df.iloc[:, -9:].sample(3, random_state=42)

,dia_do_mes,semana_do_ano,dia_semana_sin,dia_semana_cos,mes_sin,mes_cos,hora_num,hora_sin,hora_cos
data,,,,,,,,,
2024-09-23,23,39,0.000000,1.00000,-1.0,-1.836970e-16,1,0.258819,0.965926
2024-03-03,3,9,-0.781831,0.62349,1.0,6.123234e-17,0,0.000000,1.000000
2024-09-29,29,39,-0.781831,0.62349,-1.0,-1.836970e-16,11,0.258819,-0.965926


### Transformando colunas de string para float

Existem várias colunas numéricas em formatos `objects` que precisam de uma atenção. Para isso, será utilizado conversões de todas as colunas alvos para `float`

In [25]:
df.dtypes

PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                          object
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)     object
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)           object
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)          object
RADIACAO GLOBAL (Kj/m²)                                   object
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)              object
TEMPERATURA DO PONTO DE ORVALHO (°C)                      object
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)                object
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)                object
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)          object
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)          object
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                 float64
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                 float64
UMIDADE RELATIVA DO AR, HORARIA (%)                      float64
VENTO, DIREÇÃO HORARIA (gr) (° (gr))                     float64
VENTO, RAJADA MAXIMA (m/s

In [26]:
columns_to_float = df.iloc[:,0:17].columns.to_list()
df[columns_to_float] = (
    df[columns_to_float]
    .replace(r'^\s*$', np.nan, regex=True)
    .replace(",", ".", regex=True)
)

# Converter para float
df[columns_to_float] = df[columns_to_float].apply(
    pd.to_numeric, errors="coerce", downcast="float"
)

In [27]:
df.dtypes[0:11]

PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                         float32
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    float32
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          float32
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         float32
RADIACAO GLOBAL (Kj/m²)                                  float32
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)             float32
TEMPERATURA DO PONTO DE ORVALHO (°C)                     float32
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)               float32
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)               float32
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)         float32
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)         float32
dtype: object

### Lidando com valores ausentes

In [28]:
print(f"Quantidade de valor nulo por features:\n\n{df.isna().sum()}")

Quantidade de valor nulo por features:

PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                           24
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)      24
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)            24
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)           24
RADIACAO GLOBAL (Kj/m²)                                  4080
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)               26
TEMPERATURA DO PONTO DE ORVALHO (°C)                       26
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)                 26
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)                 26
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)           26
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)           26
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                   26
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                   27
UMIDADE RELATIVA DO AR, HORARIA (%)                        26
VENTO, DIREÇÃO HORARIA (gr) (° (gr))                       26
VENTO, RAJADA MAXIMA (m/s)    

Existem poucos valores nas colunas que irão ser removidas, com excessão à coluna de radiação. Remover metade das linhas iriam comprometer muitos dados importantes. Para lidar com os valores ausentes da coluna de radiação, será necessário uma inspeção rápida e ver como estão os valores presentes.

In [29]:
df[df["RADIACAO GLOBAL (Kj/m²)"].notna()].head()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),...,"VENTO, VELOCIDADE HORARIA (m/s)",dia_do_mes,semana_do_ano,dia_semana_sin,dia_semana_cos,mes_sin,mes_cos,hora_num,hora_sin,hora_cos
data,,,,,,,,,,,,,,,,,,,,,
2024-01-01,0.0,885.700012,885.799988,885.500000,4.300000,19.600000,18.799999,20.100000,19.6,19.100000,...,1.1,1,1,0.0,1.0,0.5,0.866025,9,7.071068e-01,-0.707107
2024-01-01,0.0,886.400024,886.400024,885.700012,310.100006,20.600000,19.100000,20.600000,19.6,19.200001,...,1.8,1,1,0.0,1.0,0.5,0.866025,10,5.000000e-01,-0.866025
2024-01-01,0.0,886.700012,886.700012,886.400024,779.700012,21.500000,18.500000,21.600000,20.5,19.200001,...,2.1,1,1,0.0,1.0,0.5,0.866025,11,2.588190e-01,-0.965926
2024-01-01,0.0,887.200012,887.200012,886.700012,1273.500000,22.600000,18.000000,22.900000,21.5,18.600000,...,1.8,1,1,0.0,1.0,0.5,0.866025,12,1.224647e-16,-1.000000
2024-01-01,0.0,887.400024,887.500000,887.200012,1361.099976,23.299999,17.700001,23.700001,22.5,18.799999,...,2.2,1,1,0.0,1.0,0.5,0.866025,13,-2.588190e-01,-0.965926


Podemos notar que os valores presentes da Radiação Global possuem significados válidos para algumas amostras. Levando em conta que metade dos dados são valores ausente, será necessário a aplicação de métodos cautelosos para lidar com esses dados sem perder a consistência.

Deste modo, vamos interpolar os valores com a data e preencher os valores ausentes com a média mensal.

In [30]:
# Interpolação linear na coluna de radiação
df['RADIACAO GLOBAL (Kj/m²)'] = df['RADIACAO GLOBAL (Kj/m²)'].interpolate(method='linear')

# Preencher linhas com média de cada mês
df['RADIACAO GLOBAL (Kj/m²)'] = df.groupby('dia_do_mes')['RADIACAO GLOBAL (Kj/m²)'].transform(lambda x: x.fillna(x.mean()))

In [31]:
print(f"Quantidade de valor nulo por features:\n\n{df.isna().sum()}")

Quantidade de valor nulo por features:

PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                         24
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    24
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          24
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         24
RADIACAO GLOBAL (Kj/m²)                                   0
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)             26
TEMPERATURA DO PONTO DE ORVALHO (°C)                     26
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)               26
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)               26
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)         26
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)         26
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                 26
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                 27
UMIDADE RELATIVA DO AR, HORARIA (%)                      26
VENTO, DIREÇÃO HORARIA (gr) (° (gr))                     26
VENTO, RAJADA MAXIMA (m/s)                               24


In [32]:
# dropando demais linhas ausentes
df.dropna(inplace=True)

### Renomeando colunas

O nome das features são extensos e informativos, mas é muito possível reduzir e manter a essência de cada coluna. Irei renomear as features que possuem essa necessidade. 

In [33]:
df.columns

Index(['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'RADIACAO GLOBAL (Kj/m²)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)',
       'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)',
       'VENTO, VELOCIDADE HORARIA (m/s)', 'dia_do_mes', 'semana_do_ano',
       'dia_semana_sin', 'dia_semana_cos', 'mes_sin', 'mes_cos', 'hora_num',
       'hora_sin', 'hora_cos'],
      dty

In [34]:
# Preparando dados para colunas com novos nomes
nomes_colunas = df.columns.tolist()
nomes_antigos = nomes_colunas[0:17]
nomes_novos = ["precipitacao total",
               "pressao atmos nv estacao",
               "pressao atmos max",
               "pressao atmos min",
               "radiacao",
               "temp ar",
               "temp pronto orvalho",
               "temp max",
               "temp min",
               "temp orvalho max",
               "temp orvalho min",
               "umidade max",
               "umidade min",
               "umidade relativa ar",
               "vento direcao",
               "vento rajada max",
               "vento velocidade",]

renomear_dic = dict(zip(nomes_antigos, nomes_novos))
df.rename(columns=renomear_dic, inplace=True)

### Features de lag, tendência e janela móvel

A seguir, irei criar novas features que serão de muita importância durante a modelagem dos dados. A partir das séries temporais de clima como umidade e pressão atmosférica, serão criadas variáveis de lag, tendência e janela móvel, para ajudar o modelo de Machine Learning a capturar padrões temporais que podem influenciar fenômenos, como a chuva **(que será a coluna alvo)**

In [36]:
# Criando lag da umidade
df['umidade_lag_1h'] = df['umidade relativa ar'].shift(1)
df['umidade_lag_3h'] = df['umidade relativa ar'].shift(3)

# Criando lag de pressão atmosférica
df['pressao_lag_1h'] = df['pressao atmos nv estacao'].shift(1)

# Tendência para pressão atmosférica
df['pressao_tendencia_1h'] = df['pressao atmos nv estacao'] - df['pressao_lag_1h']

print(df[['pressao atmos nv estacao', 'pressao_lag_1h', 'pressao_tendencia_1h']].head())

            pressao atmos nv estacao  pressao_lag_1h  pressao_tendencia_1h
data                                                                      
2024-01-01                885.700012             NaN                   NaN
2024-01-01                886.700012      885.700012              1.000000
2024-01-01                887.099976      886.700012              0.399963
2024-01-01                887.099976      887.099976              0.000000
2024-01-01                886.400024      887.099976             -0.699951


- **Lag** é basicamente o valor da variável em um instante anterior, neste cada à `1h` e `3h` atrás, para ajudar o modelo a lembrar do histórico recente e ainda isso pode ajudar na probabilidade da chuva.

- **Tendência** aplicada justamente na pressão atmosférica foi usada aqui para captar a instabilidade, o que pode ajudar a notar chances maiores da presença de chuva. 

In [37]:
# Definindo as janelas de tempo
janelas = [3, 6, 12, 24]

# média móvel e desvio padrão para umidade
for j in janelas:
    df[f'umidade_media_{j}h'] = df['umidade relativa ar'].rolling(window=j).mean()
    df[f'umidade_std_{j}h'] = df['umidade relativa ar'].rolling(window=j).std()

# mostrar a chuva acumulada.
df['chuva_acumulada_24h'] = df['precipitacao total'].rolling(window=24).sum()

# Visualizando o resultado
print(df[['umidade relativa ar', 'umidade_media_6h', 'umidade_std_6h']].head(10))

            umidade relativa ar  umidade_media_6h  umidade_std_6h
data                                                             
2024-01-01                 79.0               NaN             NaN
2024-01-01                 85.0               NaN             NaN
2024-01-01                 88.0               NaN             NaN
2024-01-01                 85.0               NaN             NaN
2024-01-01                 89.0               NaN             NaN
2024-01-01                 91.0         86.166667        4.215052
2024-01-01                 94.0         88.666667        3.502380
2024-01-01                 94.0         90.166667        3.544949
2024-01-01                 94.0         91.166667        3.656045
2024-01-01                 95.0         92.833333        2.316607


Por fim, essa última técnica foi aplicada para gerar features estatísticas de algumas horas alvo: 3 hors, 6 horas, 12 horas e 24 horas. Isso vai ajudar o modelo a entender além do valor pontual da variável de umidade e precipitação, como também seu comportamento recente, ou seja:

- A média móvel suavida os dados e mostra a tendência geral (como o lag ver um unico ponto, aqui é anotado um histórico recente)

- O desvio padrão é usado para medir a variabilidade da janela móvel. Isso irá ajudar a observar as mudanças da umidade, sinalizando especificamente as mudanças da massa de ar para indicar condições mais previsíveis.

- Chuva acumulada para uma observação final, um indicador direto para dizer se choveu no dia anterior.

In [38]:
df['choveu'] = (df['precipitacao total'] > 0).astype(int)
df['choveu'].value_counts()

choveu
0    8151
1     604
Name: count, dtype: int64

Para finalizar, criei as colunas alvo para treinar um modelo de classificação a partir de uma condição booleana. Se chover será marcado como 1, caso contrário, 0.

Com isso, é levantada a conclusão de que o objetivo do modelo de Machine Learning será treinado para prever se choveu ou não (modelo de classificação). Uma ideia futura poderá concretizar para prever a quantidade de mm de chuva (regressão), mas nesse momento, o foco será para a primeira opção.

In [39]:
df_final = df.dropna()
df_final.head(3)

,precipitacao total,pressao atmos nv estacao,pressao atmos max,pressao atmos min,radiacao,temp ar,temp pronto orvalho,temp max,temp min,temp orvalho max,...,umidade_media_3h,umidade_std_3h,umidade_media_6h,umidade_std_6h,umidade_media_12h,umidade_std_12h,umidade_media_24h,umidade_std_24h,chuva_acumulada_24h,choveu
data,,,,,,,,,,,,,,,,,,,,,
2024-01-01,2.6,884.500000,884.500000,883.799988,50.745457,20.500000,19.799999,20.600000,20.299999,20.000000,...,96.0,0.0,83.666667,14.841384,76.083333,14.951183,82.541667,12.765372,35.599999,1
2024-01-02,0.2,885.299988,885.299988,884.500000,45.790909,20.200001,19.600000,20.600000,20.100000,19.799999,...,96.0,0.0,86.000000,15.607690,77.833333,16.004734,83.250000,13.029231,35.799999,1
2024-01-02,0.0,885.900024,885.900024,885.299988,40.836365,20.299999,19.600000,20.299999,20.200001,19.700001,...,96.0,0.0,91.500000,11.022704,79.916667,16.648551,83.708333,13.284444,35.799999,0


### Salvando dados limpos e processados

O index é muito importante, já que foi anexado à data. Deste modo, os novos dados serão salvos em `csv` mantendo o index.

In [40]:
# inspeção final
print(f"Quantidade de dados processados: {df_final.shape[0]}")
print(f"Contagem final das features: {df_final.shape[1]}")

# salvando
diretorio_alvo = os.path.join("..\data", "INMET_DF_processado.csv")

df_final.to_csv(diretorio_alvo, encoding='utf-8', index=True)
print(f"Dados Processados salvo em: {diretorio_alvo}")

Quantidade de dados processados: 8732
Contagem final das features: 40
Dados Processados salvo em: ..\data\INMET_DF_processado.csv


A próxima etapa será realizar uma Análise Exploratória de Dados com foco na ocorrência de chuva em Brasília.

Essa análise permitirá identificar padrões sazonais e horários, relações entre variáveis climáticas e a precipitação, além de validar hipóteses como o impacto da umidade, pressão atmosférica e vento sobre a ocorrência de chuva.

Esses insights orientarão o direcionamento dos modelos de Machine Learning, garantindo que eles não apenas aprendam padrões estatísticos, mas também reflitam fenômenos meteorológicos.